In [1]:
pip install gradio

In [2]:
pip install lightly

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset_path = "/content/drive/My Drive/Datset/selected"  # Adjust to your folder structure


In [5]:
import tensorflow as tf
from tensorflow.keras import layers

# Data augmentation for SimCLR
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])


In [6]:
# Encoder Model
def create_encoder(input_shape=(224, 224, 3)):
    base_model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape=input_shape)
    encoder = tf.keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
    ])
    return encoder

encoder = create_encoder()

# Contrastive Loss for SimCLR
def contrastive_loss(projections_1, projections_2, temperature=0.1):
    projections_1 = tf.math.l2_normalize(projections_1, axis=1)
    projections_2 = tf.math.l2_normalize(projections_2, axis=1)

    # Similarity matrix
    similarity_matrix = tf.matmul(projections_1, projections_2, transpose_b=True)
    labels = tf.eye(similarity_matrix.shape[0])  # Positive pairs on diagonal
    return tf.keras.losses.CategoricalCrossentropy(from_logits=True)(labels, similarity_matrix / temperature)


In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(batch_images):
    with tf.GradientTape() as tape:
        augmented_1 = data_augmentation(batch_images)
        augmented_2 = data_augmentation(batch_images)

        projections_1 = encoder(augmented_1, training=True)
        projections_2 = encoder(augmented_2, training=True)

        loss = contrastive_loss(projections_1, projections_2)

    gradients = tape.gradient(loss, encoder.trainable_variables)
    optimizer.apply_gradients(zip(gradients, encoder.trainable_variables))
    return loss


In [ ]:
# Load dataset and preprocess
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return img_array

import os
import numpy as np

def load_dataset(image_folder):
    images = []
    for image_name in os.listdir(image_folder):
        if image_name.endswith((".jpg", ".jpeg", ".png")):
            images.append(preprocess_image(os.path.join(image_folder, image_name)))
    return np.array(images)

images = load_dataset(dataset_path)
batch_size = 32
epochs = 10

for epoch in range(epochs):
    total_loss = 0
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i+batch_size]
        loss = train_step(batch_images)
        total_loss += loss.numpy()
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 19.6442
Epoch 2, Loss: 13.1485
Epoch 3, Loss: 10.9233
Epoch 4, Loss: 9.9156


In [ ]:
# Define the encoder model
encoder = create_encoder(input_shape=(224, 224, 3))

# Build the encoder model by calling it with a dummy input
dummy_input = tf.keras.Input(shape=(224, 224, 3))
encoder_output = encoder(dummy_input)


In [ ]:
from tensorflow.keras import layers, Model

# Attach classification head
x = layers.Dense(128, activation="relu")(encoder_output)
x = layers.Dropout(0.5)(x)
output = layers.Dense(3, activation="softmax")(x)  # 3 classes: Low, Medium, High

# Build the final model
model = Model(inputs=dummy_input, outputs=output)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load pre-trained ResNet50 without the top classification layer
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))

def extract_features(image_folder, img_size=(224, 224)):
    """
    Extracts features from images in a folder using a pre-trained model.
    Args:
        image_folder (str): Path to the folder containing images.
        img_size (tuple): Size to resize images.
    Returns:
        np.ndarray: Extracted features.
        list: List of image file paths.
    """
    features = []
    image_paths = []

    for image_name in os.listdir(image_folder):
        if image_name.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(image_folder, image_name)
            img = load_img(image_path, target_size=img_size)
            img_array = img_to_array(img) / 255.0  # Normalize pixel values
            img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

            # Extract features
            feature = base_model.predict(img_array)
            features.append(feature.flatten())
            image_paths.append(image_path)

    return np.array(features), image_paths

# Path to the dataset (adjust to your directory)
dataset_path = "/content/drive/My Drive/Datset/temp"
features, image_paths = extract_features(dataset_path)
print(f"Extracted features shape: {features.shape}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Extracted features shape: (10, 2048)


In [ ]:
from sklearn.cluster import KMeans

# Apply K-Means clustering
num_clusters = 3  # Assume 3 clusters for Low, Medium, High Risk
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(features)

# Map each image to a cluster
for path, cluster in zip(image_paths, clusters):
    print(f"Image: {path}, Cluster: {cluster}")


Image: /content/drive/My Drive/Datset/temp/Copy of 8.png, Cluster: 0
Image: /content/drive/My Drive/Datset/temp/Copy of 3.png, Cluster: 2
Image: /content/drive/My Drive/Datset/temp/Copy of 1.png, Cluster: 0
Image: /content/drive/My Drive/Datset/temp/Copy of 9.png, Cluster: 0
Image: /content/drive/My Drive/Datset/temp/Copy of 5.png, Cluster: 2
Image: /content/drive/My Drive/Datset/temp/Copy of 7.png, Cluster: 0
Image: /content/drive/My Drive/Datset/temp/Copy of 2.png, Cluster: 0
Image: /content/drive/My Drive/Datset/temp/Copy of 4.png, Cluster: 1
Image: /content/drive/My Drive/Datset/temp/Copy of 6.png, Cluster: 1
Image: /content/drive/My Drive/Datset/temp/Copy of 10.png, Cluster: 0


In [ ]:
import shutil

output_path = "/content/drive/My Drive/clustered_images"
for path, cluster in zip(image_paths, clusters):
    cluster_folder = os.path.join(output_path, str(cluster))
    os.makedirs(cluster_folder, exist_ok=True)
    shutil.copy(path, cluster_folder)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50

# Data Generators
datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    output_path, target_size=(224, 224), batch_size=16, class_mode="categorical", subset="training"
)
val_gen = datagen.flow_from_directory(
    output_path, target_size=(224, 224), batch_size=16, class_mode="categorical", subset="validation"
)

# Load pre-trained ResNet50 and attach classification head
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))
base_model.trainable = False

# Add classification layers
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)  # 3 clusters
model = Model(inputs, outputs)

# Compile and train
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_gen, validation_data=val_gen, epochs=10)

# Save the fine-tuned model
model.save("pseudo_label_model.h5")


Found 9 images belonging to 3 classes.
Found 1 images belonging to 3 classes.
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.2222 - loss: 1.8669 - val_accuracy: 0.0000e+00 - val_loss: 1.1585
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4444 - loss: 1.3172 - val_accuracy: 1.0000 - val_loss: 0.4786
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4444 - loss: 1.1700 - val_accuracy: 1.0000 - val_loss: 0.2428
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 824ms/step - accuracy: 0.6667 - loss: 1.2399 - val_accuracy: 1.0000 - val_loss: 0.1669
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 806ms/step - accuracy: 0.5556 - loss: 1.3733 - val_accuracy: 1.0000 - val_loss: 0.1580
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5556 - loss: 1.3697 - val_accuracy: 1.0000 - val_loss: 0.1908
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5556 - loss: 1.5794 - val_accuracy: 1.0000 - val_loss: 0.2606
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 822ms/step - accuracy: 0.5556 - loss: 1.4024 - val_accuracy: 1.0000 - val_loss: 0.3607
Epoch 9/10
1

In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

# Load the trained model
model = load_model("pseudo_label_model.h5")  # Replace with your model path

# Define the prediction function
def predict_retina_image(image):
    """
    Predict cardiovascular risk from a retina image.
    Args:
        image (PIL.Image): Input retina image uploaded by the user.
    Returns:
        str: Predicted risk and confidence scores.
    """
    try:
        # Define cluster names
        cluster_names = {0: "Low Risk", 1: "Medium Risk", 2: "High Risk"}

        # Preprocess the image
        image = image.resize((224, 224))  # Resize to match model input
        img_array = np.array(image) / 255.0  # Normalize pixel values
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Predict
        predictions = model.predict(img_array)
        predicted_cluster = np.argmax(predictions)  # Get the cluster with the highest probability
        cluster_name = cluster_names[predicted_cluster]  # Map cluster to name

        # Confidence scores
        confidence_scores = {
            cluster_names[i]: round(float(score) * 100, 2)
            for i, score in enumerate(predictions[0])
        }

        # Return results
        return f"Predicted Risk: {cluster_name}\nConfidence Scores: {confidence_scores}"

    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
interface = gr.Interface(
    fn=predict_retina_image,  # The function to call
    inputs=gr.Image(type="pil", label="Upload Retina Image"),  # Input: Retina image
    outputs=gr.Textbox(label="Prediction"),  # Output: Textbox for results
    title="Cardiovascular Risk Prediction",
    description="Upload a retina image to predict the cardiovascular risk cluster (Low Risk, Medium Risk, High Risk).",
)

# Launch the Gradio app
if __name__ == "__main__":
    interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://93b483cd12ba27f9c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
